In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

In [24]:
df = pd.read_csv("../data/cleaned.csv")
df.head()

,Unnamed: 0,AGE,SEX,RACE,MARST,EDUC,STATEFIP,INCWAGE,CITIZEN,INCWAGE_LOG
0,842,49,2,6,6,6,1,7000.0,1,8.853665
1,1125,49,2,6,6,6,1,7000.0,1,8.853665
2,1875,49,2,6,6,6,1,7000.0,1,8.853665
3,2945,53,2,7,1,3,1,37400.0,1,10.529426
4,2946,46,1,7,1,0,1,360.0,0,5.886104


In [25]:
model = smf.ols(
    formula=' INCWAGE_LOG ~ CITIZEN + AGE + C(SEX) + C(RACE) + C(MARST) + C(EDUC) + C(STATEFIP)',
    data=df
).fit(cov_type='HC3')  # robust SEs

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            INCWAGE_LOG   R-squared:                       0.258
Model:                            OLS   Adj. R-squared:                  0.257
Method:                 Least Squares   F-statistic:                     1011.
Date:                Mon, 01 Dec 2025   Prob (F-statistic):               0.00
Time:                        19:31:04   Log-Likelihood:            -3.0597e+05
No. Observations:              223823   AIC:                         6.121e+05
Df Residuals:                  223746   BIC:                         6.129e+05
Df Model:                          76                                         
Covariance Type:                  HC3                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             9.9277      0.03

Outcome regression suggests a 21% wage premium for naturalized immigrants. Because naturalization is not randomly assigned, this may reflect both causal effects and unobserved selection. Therefore, we next apply causal methods (e.g., IPW, AIPW, overlap weighting) to obtain more credible treatment effects.

### Logistic Regression Propensity Score

In [26]:
df_lr = df.copy()
prop_formula = "CITIZEN ~ AGE + C(SEX) + C(RACE) + C(MARST) + C(EDUC) + C(STATEFIP)"

prop_model = smf.logit(prop_formula, data=df_lr).fit(disp=False)
df_lr["e_hat"] = prop_model.predict(df)
df_lr["e_hat"] = df_lr["e_hat"].clip(0.01, 0.99)
df_lr.head()

,Unnamed: 0,AGE,SEX,RACE,MARST,EDUC,STATEFIP,INCWAGE,CITIZEN,INCWAGE_LOG,e_hat
0,842,49,2,6,6,6,1,7000.0,1,8.853665,0.640226
1,1125,49,2,6,6,6,1,7000.0,1,8.853665,0.640226
2,1875,49,2,6,6,6,1,7000.0,1,8.853665,0.640226
3,2945,53,2,7,1,3,1,37400.0,1,10.529426,0.258104
4,2946,46,1,7,1,0,1,360.0,0,5.886104,0.203977


In [27]:
outcome_formula = "INCWAGE_LOG ~ AGE + C(SEX) + C(RACE) + C(MARST) + C(EDUC) + C(STATEFIP)"
m1_model = smf.ols(outcome_formula, data=df_lr[df_lr["CITIZEN"] == 1]).fit()
m0_model = smf.ols(outcome_formula, data=df_lr[df_lr["CITIZEN"] == 0]).fit()
df_lr["m1_hat"] = m1_model.predict(df)
df_lr["m0_hat"] = m0_model.predict(df)
T = df_lr["CITIZEN"].values
Y = df_lr["INCWAGE_LOG"].values
e = df_lr["e_hat"].values
m1 = df_lr["m1_hat"].values
m0 = df_lr["m0_hat"].values
psi = (
    m1 - m0
    + T * (Y - m1) / e
    - (1 - T) * (Y - m0) / (1 - e)
)

tau_hat = psi.mean()
se_hat = psi.std(ddof=1) / np.sqrt(len(psi))

print(f"AIPW estimate (log-wage scale): {tau_hat:.4f}")
print(f"Std. error: {se_hat:.4f}")

AIPW estimate (log-wage scale): 0.1852
Std. error: 0.0047


In [29]:
perc = 100 * (np.exp(tau_hat) - 1)

ci_low = tau_hat - 1.96 * se_hat
ci_high = tau_hat + 1.96 * se_hat
perc_low = 100 * (np.exp(ci_low) - 1)
perc_high = 100 * (np.exp(ci_high) - 1)

print(f"AIPW effect ≈ {perc:.1f}% change in wages")
print(f"95% CI: [{perc_low:.1f}%, {perc_high:.1f}%]")


AIPW effect ≈ 20.3% change in wages
95% CI: [19.2%, 21.5%]


Using an augmented inverse probability weighting estimator that combines a propensity-score model and separate outcome regressions for naturalized and non-citizen immigrants, we estimate that naturalization increases wages by about 20.3% (95% CI [19.2%, 21.5%]) on average.

### Random Forest Propensity Score

In [30]:
df_rf = df.copy()
df_rf.head()

,Unnamed: 0,AGE,SEX,RACE,MARST,EDUC,STATEFIP,INCWAGE,CITIZEN,INCWAGE_LOG
0,842,49,2,6,6,6,1,7000.0,1,8.853665
1,1125,49,2,6,6,6,1,7000.0,1,8.853665
2,1875,49,2,6,6,6,1,7000.0,1,8.853665
3,2945,53,2,7,1,3,1,37400.0,1,10.529426
4,2946,46,1,7,1,0,1,360.0,0,5.886104


In [16]:
from sklearn.ensemble import RandomForestClassifier

cat_vars = ["SEX", "RACE", "MARST", "EDUC", "STATEFIP"]
X = pd.get_dummies(df_rf[["AGE"] + cat_vars], drop_first=True)
T = df_rf["CITIZEN"].values
rf = RandomForestClassifier(
    n_estimators=500,
    max_depth=None,
    min_samples_leaf=20,
    random_state=0,
    n_jobs=-1
)
rf.fit(X, T)
df_rf["e_hat"] = rf.predict_proba(X)[:, 1]
df_rf["e_hat"] = df_rf["e_hat"].clip(0.01, 0.99)

In [ ]:

outcome_formula = "INCWAGE_LOG ~ AGE + C(SEX) + C(RACE) + C(MARST) + C(EDUC) + C(STATEFIP)"

m1_model = smf.ols(outcome_formula, data=df_rf[df_rf["CITIZEN"] == 1]).fit()
m0_model = smf.ols(outcome_formula, data=df_rf[df_rf["CITIZEN"] == 0]).fit()

df_rf["m1_hat"] = m1_model.predict(df)
df_rf["m0_hat"] = m0_model.predict(df)


In [21]:
T = df_rf["CITIZEN"].values
Y = df_rf["INCWAGE_LOG"].values
e = df_rf["e_hat"].values
m1 = df_rf["m1_hat"].values
m0 = df_rf["m0_hat"].values

psi = (
    m1 - m0
    + T * (Y - m1) / e
    - (1 - T) * (Y - m0) / (1 - e)
)

tau_hat = psi.mean()
se_hat = psi.std(ddof=1) / np.sqrt(len(psi))

print(f"AIPW (RF propensity) on log-wage: {tau_hat:.4f}")
print(f"Std. error: {se_hat:.4f}")


AIPW (RF propensity) on log-wage: 0.2172
Std. error: 0.0045


In [22]:
perc = 100 * (np.exp(tau_hat) - 1)

ci_low = tau_hat - 1.96 * se_hat
ci_high = tau_hat + 1.96 * se_hat
perc_low = 100 * (np.exp(ci_low) - 1)
perc_high = 100 * (np.exp(ci_high) - 1)

print(f"AIPW effect ≈ {perc:.1f}% change in wages")
print(f"95% CI: [{perc_low:.1f}%, {perc_high:.1f}%]")

AIPW effect ≈ 24.3% change in wages
95% CI: [23.2%, 25.4%]


### Simple Neural Network

In [31]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
df_nn = df.copy()
cat_vars = ["SEX", "RACE", "MARST", "EDUC", "STATEFIP"]
X_cat = pd.get_dummies(df_nn[cat_vars], drop_first=True)
X_cont = df_nn[["AGE"]].copy()
scaler = StandardScaler()
X_cont_scaled = pd.DataFrame(
    scaler.fit_transform(X_cont),
    columns=X_cont.columns,
    index=X_cont.index
)
X = pd.concat([X_cont_scaled, X_cat], axis=1)
T = df_nn["CITIZEN"].values


In [32]:
rf_seed = 42 
mlp = MLPClassifier(
    hidden_layer_sizes=(32,),
    solver="adam",
    alpha=1e-4,
    batch_size="auto",
    learning_rate="adaptive",
    max_iter=200,
    random_state=rf_seed,
    early_stopping=True,
    n_iter_no_change=5,
    verbose=False
)

mlp.fit(X, T)

# Predicted propensity scores P(T=1|X)
df_nn["e_hat"] = mlp.predict_proba(X)[:, 1]

# Clip to avoid extreme weights
df_nn["e_hat"] = df_nn["e_hat"].clip(0.01, 0.99)


In [33]:

outcome_formula = "INCWAGE_LOG ~ AGE + C(SEX) + C(RACE) + C(MARST) + C(EDUC) + C(STATEFIP)"

m1_model = smf.ols(outcome_formula, data=df_nn[df_nn["CITIZEN"] == 1]).fit()
m0_model = smf.ols(outcome_formula, data=df_nn[df_nn["CITIZEN"] == 0]).fit()

df_nn["m1_hat"] = m1_model.predict(df)
df_nn["m0_hat"] = m0_model.predict(df)


In [34]:
T = df_nn["CITIZEN"].values
Y = df_nn["INCWAGE_LOG"].values
e = df_nn["e_hat"].values
m1 = df_nn["m1_hat"].values
m0 = df_nn["m0_hat"].values

psi = (
    m1 - m0
    + T * (Y - m1) / e
    - (1 - T) * (Y - m0) / (1 - e)
)

tau_hat = psi.mean()
se_hat = psi.std(ddof=1) / np.sqrt(len(psi))

print(f"AIPW (RF propensity) on log-wage: {tau_hat:.4f}")
print(f"Std. error: {se_hat:.4f}")

AIPW (RF propensity) on log-wage: 0.2219
Std. error: 0.0046


In [35]:
perc = 100 * (np.exp(tau_hat) - 1)

ci_low = tau_hat - 1.96 * se_hat
ci_high = tau_hat + 1.96 * se_hat
perc_low = 100 * (np.exp(ci_low) - 1)
perc_high = 100 * (np.exp(ci_high) - 1)

print(f"AIPW effect ≈ {perc:.1f}% change in wages")
print(f"95% CI: [{perc_low:.1f}%, {perc_high:.1f}%]")

AIPW effect ≈ 24.8% change in wages
95% CI: [23.7%, 26.0%]
